In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008}
band = [6]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.66, 1.66, 1.85)]
  wgt = [(0.5, 0.55, 0.85)]
o co2 band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.001169                    1 -0.000997
0.000750        2 -0.001169                    2 -0.001016
0.001052        3 -0.001169                    3 -0.000991
0.001476        4 -0.001168                    4 -0.000987
0.002070        5 -0.001166                    5 -0.000987
0.002904        6 -0.001163                    6 -0.000984
0.004074        7 -0.001159                    7 -0.000981
0.005714        8 -0.001153                    8 -0.000975
0.008015        9 -0.001146                    9 -0.000970
0.011243       10 -0.001137                   10 -0.000965
0.015771       11 -0.001127                   11 -0.000964
0.022122       12 -0.001116                   12 -0.000960
0.031031       13 -0.001104                   13 -0.000953
0.043528       14 -0.001088                   14 -0.000945
0.061057       15 -0.001028                   15 -0.000908
0.085645       16 -0.000754                   16 -0.000726
0.120136       17  0.000022                   17 -0.000196
0.168516       18  0.001521                   18  0.000841
0.236378       19  0.004106                   19  0.002621
0.331549       20  0.008282                   20  0.005478
0.465100       21  0.014715                   21  0.009937
0.652400       22  0.024225                   22  0.017165
0.915100       23  0.034405                   23  0.025059
1.283650       24  0.034360                   24  0.025474
1.800600       25  0.024782                   25  0.018983
2.525700       26  0.015777                   26  0.012546
3.542800       27  0.009294                   27  0.007627
4.969550       28  0.004985                   28  0.004174
6.970850       29  0.002300                   29  0.001871
9.778100       30  0.000643                   30  0.000347
13.715850      31 -0.000420                   31 -0.000690
19.239350      32 -0.001076                   32 -0.001355
26.987250      33 -0.001422                   33 -0.001730
37.855300      34 -0.001615                   34 -0.001953
53.100050      35 -0.001766                   35 -0.002106
73.887500      36 -0.001891                   36 -0.002212
97.662500      37 -0.001998                   37 -0.002296
121.437500     38 -0.002104                   38 -0.002353
145.212500     39 -0.002204                   39 -0.002343
168.987500     40 -0.002313                   40 -0.002395
192.762500     41 -0.002466                   41 -0.002491
216.537500     42 -0.002608                   42 -0.002725
240.312500     43 -0.002643                   43 -0.002865
264.087500     44 -0.002526                   44 -0.002872
287.862500     45 -0.002178                   45 -0.002711
311.637500     46 -0.001543                   46 -0.002292
335.412500     47 -0.000595                   47 -0.001560
359.187500     48  0.000733                   48 -0.000517
382.962500     49  0.002504                   49  0.000927
406.737500     50  0.004774                   50  0.002854
430.512500     51  0.007585                   51  0.005299
454.287500     52  0.010809                   52  0.008239
478.062500     53  0.014481                   53  0.011740
501.837500     54  0.018697                   54  0.015891
525.612500     55  0.023495                   55  0.020539
549.387500     56  0.028915                   56  0.025910
573.162500     57  0.035017                   57  0.032059
596.937500     58  0.043226                   58  0.040301
620.712500     59  0.052523                   59  0.049770
644.487500     60  0.062573                   60  0.059875
668.262500     61  0.073341                   61  0.070517
692.037500     62  0.084747                   62  0.081731
715.812500     63  0.096801                   63  0.093607
739.587500     64  0.109516                   64  0.106250
763.362500     65  0.122867                   65  0.119820
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -59.789557  0.000000e+00 -59.789557 -59.848948   
0.000624    2     -59.789557  6.389750e-08 -59.789557 -59.848948   
0.000876    3     -59.789557  9.008948e-08 -59.789557 -59.848948   
0.001229    4     -59.789557  1.282095e-07 -59.789557 -59.848948   
0.001723    5     -59.789557  1.838239e-07 -59.789557 -59.848948   
0.002417    6     -59.789557  2.649369e-07 -59.789557 -59.848949   
0.003391    7     -59.789558  3.832374e-07 -59.789557 -59.848949   
0.004757    8     -59.789558  5.557579e-07 -59.789557 -59.848949   
0.006672    9     -59.789559  8.073012e-07 -59.789558 -59.848950   
0.009359    10    -59.789559  1.174031e-06 -59.789558 -59.848950   
0.013128    11    -59.789560  1.708620e-06 -59.789559 -59.848951   
0.018415    12    -59.789562  2.487815e-06 -59.789559 -59.848952   
0.025830    13    -59.789564  3.623399e-06 -59.789560 -59.848954   
0.036232    14    -59.789567  5.278229e-06 -59.789562 -59.848956   
0.050823    15    -59.789571  7.695435e-06 -59.789564 -59.848960   
0.071291    16    -59.789578  1.151210e-05 -59.789566 -59.848965   
0.100000    17    -59.789587  1.880909e-05 -59.789569 -59.848973   
0.140271    18    -59.789603  3.445135e-05 -59.789568 -59.848985   
0.196760    19    -59.789626  6.777383e-05 -59.789558 -59.849004   
0.275997    20    -59.789657  1.377140e-04 -59.789520 -59.849031   
0.387100    21    -59.789693  2.820668e-04 -59.789411 -59.849062   
0.543100    22    -59.789714  5.749780e-04 -59.789139 -59.849089   
0.761700    23    -59.789667  1.155523e-03 -59.788512 -59.849081   
1.068500    24    -59.789461  2.199294e-03 -59.787261 -59.848986   
1.498800    25    -59.789126  3.615924e-03 -59.785510 -59.848840   
2.102400    26    -59.788835  5.096857e-03 -59.783738 -59.848775   
2.949000    27    -59.788678  6.522032e-03 -59.782156 -59.848865   
4.136600    28    -59.788711  7.862964e-03 -59.780848 -59.849161   
5.802500    29    -59.788991  9.126308e-03 -59.779865 -59.849718   
8.139200    30    -59.789579  1.035123e-02 -59.779228 -59.850600   
11.417000   31    -59.790563  1.158509e-02 -59.778978 -59.851898   
16.014700   32    -59.792063  1.285595e-02 -59.779207 -59.853732   
22.464000   33    -59.794225  1.419586e-02 -59.780029 -59.856252   
31.510500   34    -59.797272  1.571866e-02 -59.781553 -59.859709   
44.200100   35    -59.801571  1.759077e-02 -59.783980 -59.864517   
62.000000   36    -59.807621  1.991798e-02 -59.787703 -59.871226   
85.775000   37    -59.815724  2.269585e-02 -59.793028 -59.880158   
109.550000  38    -59.824038  2.538150e-02 -59.798656 -59.889247   
133.325000  39    -59.832724  2.814229e-02 -59.804581 -59.898546   
157.100000  40    -59.841750  3.096334e-02 -59.810787 -59.907802   
180.875000  41    -59.851278  3.397594e-02 -59.817302 -59.917357   
204.650000  42    -59.861992  3.774321e-02 -59.824248 -59.927776   
228.425000  43    -59.874671  4.307882e-02 -59.831592 -59.940290   
252.200000  44    -59.889670  5.063297e-02 -59.839037 -59.955182   
275.975000  45    -59.907363  6.121251e-02 -59.846151 -59.972694   
299.750000  46    -59.928289  7.600403e-02 -59.852285 -59.993273   
323.525000  47    -59.952304  9.567418e-02 -59.856630 -60.016852   
347.300000  48    -59.979355  1.210511e-01 -59.858304 -60.043080   
371.075000  49    -60.009599  1.533597e-01 -59.856239 -60.072248   
394.850000  50    -60.043186  1.939992e-01 -59.849187 -60.104686   
418.625000  51    -60.080245  2.445030e-01 -59.835742 -60.140432   
442.400000  52    -60.120901  3.065212e-01 -59.814380 -60.179398   
466.175000  53    -60.164888  3.809484e-01 -59.783939 -60.221527   
489.950000  54    -60.212063  4.689083e-01 -59.743155 -60.266822   
513.725000  55    -60.262436  5.719369e-01 -59.690499 -60.315400   
537.500000  56    -60.315994  6.916631e-01 -59.624330 -60.366768   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')